In [1]:
import numpy as np
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
import pickle

Using TensorFlow backend.


# Check if tf-gpu is enabled

If not : 
and
tg-gpu=2.1.0
keras : conda keras=2.3.1

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

tf.__version__

Default GPU Device:/device:GPU:0


'2.1.0'

In [3]:
import data_utils
import glove_utils
import models
import display_utils
from goog_lm import LM

In [4]:
import lm_data_utils
import lm_utils

In [5]:
np.random.seed(1002)
tf.compat.v1.set_random_seed
(1002)

1002

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
VOCAB_SIZE  = 50000
with open('aux_files/dataset_%d.pkl' %VOCAB_SIZE, 'rb') as f:
    dataset = pickle.load(f)

In [8]:
doc_len = [len(dataset.test_seqs2[i]) for i in 
           range(len(dataset.test_seqs2))]

In [9]:
dist_mat = np.load('aux_files/dist_counter_%d.npy' %VOCAB_SIZE)
# Prevent returning 0 as most similar word because it is not part of the dictionary
dist_mat[0,:] = 100000
dist_mat[:,0] = 100000

skip_list = np.load('aux_files/missed_embeddings_counter_%d.npy' %VOCAB_SIZE)

### Demonstrating how we find the most similar words

In [10]:
for i in range(300, 305):
    src_word = i
    nearest, nearest_dist = glove_utils.pick_most_similar_words(src_word, dist_mat,20, 0.5)
        
    print('Closest to `%s` are:' %(dataset.inv_dict[src_word]))
    for w_id, w_dist in zip(nearest, nearest_dist):
          print(' -- ', dataset.inv_dict[w_id], ' ', w_dist)

    print('----')

Closest to `later` are:
 --  subsequent   0.1832310977140006
 --  subsequently   0.18671959913400138
 --  afterward   0.2509214012219996
 --  afterwards   0.2576958961479985
 --  thereafter   0.27419810965900004
 --  trailing   0.336800271281001
 --  after   0.3452026123779999
 --  then   0.3647283933829992
 --  posterior   0.4310855888389993
 --  following   0.48330736760400095
----
Closest to `takes` are:
 --  pick   0.3113054656320011
 --  taking   0.42471158462800096
 --  picked   0.4852741249590009
----
Closest to `instead` are:
 --  conversely   0.3034038049850012
 --  however   0.3475382865829997
 --  alternatively   0.39540487543000036
 --  alternately   0.4439627395600003
 --  nevertheless   0.47716397579200054
----
Closest to `seem` are:
 --  seems   0.007052995653000105
 --  appears   0.32837244735200044
 --  looks   0.33534638306400066
 --  transpires   0.45620718549300143
----
Closest to `john` are:
 --  jon   0.266846971000001
 --  giovanni   0.28136114290399905
 --  joha

### Preparing the dataset

In [11]:
max_len = 250
train_x = pad_sequences(dataset.train_seqs2, maxlen=max_len, padding='post')
train_y = np.array(dataset.train_y)
test_x = pad_sequences(dataset.test_seqs2, maxlen=max_len, padding='post')
test_y = np.array(dataset.test_y)

### Loading the sentiment analysis model

In [12]:
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()
if tf.compat.v1.get_default_session():
    sess.close()
sess = tf.compat.v1.Session()
batch_size = 1
lstm_size = 128
#max_len =  100

with tf.compat.v1.variable_scope('imdb', reuse=False):
    model = models.SentimentModel(batch_size=batch_size,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
saver = tf.compat.v1.train.Saver()
saver.restore(sess, './models/imdb_model_dropout_stop5')



Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from ./models/imdb_model_dropout_stop5


## Loading the Google Language model

In [13]:
goog_lm = LM()

goog_lm/vocab-2016-09-10.txt
LM vocab loading done
Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:42:00.0, compute capability: 6.1



Recovering graph.


INFO:tensorflow:Recovering Graph goog_lm/graph-2016-09-10.pbtxt


Recovering checkpoint goog_lm/ckpt-*


#### demonstrating the GoogLM

In [14]:
src_word = dataset.dict['play']
nearest, nearest_dist = glove_utils.pick_most_similar_words(src_word, dist_mat,20)
nearest_w = [dataset.inv_dict[x] for x in nearest]
print('Closest to `%s` are %s' %(dataset.inv_dict[src_word], nearest_w))

Closest to `play` are ['playing', 'gaming', 'games', 'toy', 'playback', 'game', 'plaything', 'cheek', 'gambling', 'toys', 'toying', 'replay', 'stake', 'plays', 'jeu', 'gamble', 'staking', 'reproduction', 'casino', 'sets']


In [15]:
prefix = 'is'
suffix = 'with'
lm_preds = goog_lm.get_words_probs(prefix, nearest_w, suffix)
print('most probable is ', nearest_w[np.argmax(lm_preds)])


InternalError: 2 root error(s) found.
  (0) Internal: Blas GEMM launch failed : a.shape=(1, 4096), b.shape=(4096, 4096), m=1, n=4096, k=4096
	 [[node char_embedding/dense_0/tr/MatMul (defined at C:\Users\uros\Desktop\game_theory\nlp_adversarial_examples\lm_utils.py:46) ]]
  (1) Internal: Blas GEMM launch failed : a.shape=(1, 4096), b.shape=(4096, 4096), m=1, n=4096, k=4096
	 [[node char_embedding/dense_0/tr/MatMul (defined at C:\Users\uros\Desktop\game_theory\nlp_adversarial_examples\lm_utils.py:46) ]]
	 [[char_embedding/dense_1/tr/Relu/_159]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'char_embedding/dense_0/tr/MatMul':
  File "C:\Users\uros\.conda\envs\game3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\uros\.conda\envs\game3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\uros\.conda\envs\game3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_

## Try Attack

In [ ]:
from attacks import GeneticAtack

## Main Attack 

In [ ]:
pop_size = 60
n1 = 8

with tf.compat.v1.variable_scope('imdb', reuse=True):
    batch_model = models.SentimentModel(batch_size=pop_size,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
    
with tf.compat.v1.variable_scope('imdb', reuse=True):
    neighbour_model = models.SentimentModel(batch_size=n1,
                           lstm_size = lstm_size,
                           max_len = max_len,
                           embeddings_dim=300, vocab_size=dist_mat.shape[1],is_train = False)
ga_atttack = GeneticAtack(sess, model, batch_model, neighbour_model, dataset, dist_mat, 
                                  skip_list,
                                  goog_lm, max_iters=30, 
                                   pop_size=pop_size,
                                  
                                  n1 = n1,
                                  n2 = 4,
                                 use_lm = True, use_suffix=False)

In [ ]:
np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)

In [ ]:
import past_queries

past_queries.index = 0
past_queries.queries = np.ones((100,300),dtype=np.uint8)*-1
past_queries.outputs = np.ones((100,2), dtype=np.float32)*-1


SAMPLE_SIZE = 5000
#TEST_SIZE = 200
TEST_SIZE=2
test_idx = np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
test_len = []
for i in range(SAMPLE_SIZE):
    test_len.append(len(dataset.test_seqs2[test_idx[i]]))
print('Shortest sentence in our test set is %d words' %np.min(test_len))

test_list = []
orig_list = []
orig_label_list = []
adv_list = []
dist_list = []

for i in range(SAMPLE_SIZE):
    x_orig = test_x[test_idx[i]]
    orig_label = test_y[test_idx[i]]
    orig_preds=  model.predict(sess, x_orig[np.newaxis, :])[0]
    # print(orig_label, orig_preds, np.argmax(orig_preds))
    if np.argmax(orig_preds) != orig_label:
        #print('skipping wrong classifed ..')
        #print('--------------------------')
        continue
    x_len = np.sum(np.sign(x_orig))
    if x_len >= 100:
        #print('skipping too long input..')
        #print('--------------------------')
        continue
    # if np.max(orig_preds) < 0.90:
    #    print('skipping low confidence .. \n-----\n')
    #    continue
    print('****** ', len(test_list) + 1, ' ********')
    test_list.append(test_idx[i])
    orig_list.append(x_orig)
    target_label = 1 if orig_label == 0 else 0
    orig_label_list.append(orig_label)
    x_adv = ga_atttack.attack( x_orig, target_label)
    adv_list.append(x_adv)
    if x_adv is None:
        print('%d failed' %(i+1))
        dist_list.append(100000)
    else:
        num_changes = np.sum(x_orig != x_adv)
        print('%d - %d changed.' %(i+1, num_changes))
        dist_list.append(num_changes)
        # display_utils.visualize_attack(sess, model, dataset, x_orig, x_adv)
    print('--------------------------')
    if (len(test_list)>= TEST_SIZE):
        break

Shortest sentence in our test set is 6 words
******  1  ********
		 0  --  0.012224255
		 1  --  0.011246832
		 2  --  0.022205131
		 3  --  0.024045613
		 4  --  0.026330853
		 5  --  0.035664536
		 6  --  0.05825669
		 7  --  0.0650182
		 8  --  0.061665986
		 9  --  0.061665986
		 10  --  0.09524163
		 11  --  0.09524163
		 12  --  0.11063375
		 13  --  0.11063375
		 14  --  0.15193066
		 15  --  0.15193066


In [20]:
import defense_variables

defense_variables.index_queries = 0
defense_variables.past_queries = np.ones((100,300),dtype=np.uint8)*-1



SAMPLE_SIZE = 5000
#TEST_SIZE = 200
TEST_SIZE=100
test_idx = np.random.choice(len(dataset.test_y), SAMPLE_SIZE, replace=False)
test_len = []
for i in range(SAMPLE_SIZE):
    test_len.append(len(dataset.test_seqs2[test_idx[i]]))
print('Shortest sentence in our test set is %d words' %np.min(test_len))

test_list = []
orig_list = []
orig_label_list = []
adv_list = []
dist_list = []

for i in range(TEST_SIZE):
    x_orig = test_x[test_idx[i]]
    #print(x_orig)
    orig_label = test_y[test_idx[i]]
    x_len = np.sum(np.sign(x_orig))
    orig_list = list(x_orig[:x_len])
    #print(dataset.build_text(orig_list))
    #print("**************************************************")
    orig_preds=  model.predict(sess, x_orig[np.newaxis, :])[0]

Shortest sentence in our test set is 9 words


KeyboardInterrupt: 

## Compute Attack success rate

In [34]:
orig_len = [np.sum(np.sign(x)) for x in orig_list]
normalized_dist_list = [dist_list[i]/orig_len[i] for i in range(len(orig_list)) ]

In [35]:
SUCCESS_THRESHOLD  = 0.25
successful_attacks = [x < SUCCESS_THRESHOLD for x in normalized_dist_list]
print('Attack success rate : {:.2f}%'.format(np.mean(successful_attacks)*100))
print('Median percentange of modifications: {:.02f}% '.format(
    np.median([x for x in normalized_dist_list if x < 1])*100))
print('Mean percentange of modifications: {:.02f}% '.format(
    np.mean([x for x in normalized_dist_list if x < 1])*100))

Attack success rate : 88.00%
Median percentange of modifications: 8.93% 
Mean percentange of modifications: 11.31% 


In [36]:
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(sess, model, dataset, orig_list[visual_idx], adv_list[visual_idx])

Original Prediction = Positive. (Confidence = 80.18) 


---------  After attack -------------
New Prediction = Negative. (Confidence = 50.89) 


In [37]:
visual_idx = np.random.choice(len(orig_list))
display_utils.visualize_attack(sess, model, dataset, orig_list[visual_idx], adv_list[visual_idx])

Original Prediction = Negative. (Confidence = 86.65) 


---------  After attack -------------
New Prediction = Positive. (Confidence = 60.90) 


In [38]:
## Save success
with open('attack_results_final_dropout.pkl', 'wb') as f:
    pickle.dump((test_list, orig_list, orig_label_list, adv_list, normalized_dist_list), f)
    
